##### Copyright 2023 Google LLC

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Setup

In [1]:
!pip install -U -q "google-generativeai>=0.8.2"

In [2]:
# import necessary modules.
import base64
import copy
import json
import pathlib
import requests


import PIL.Image
import IPython.display
from IPython.display import Markdown

try:
    # The SDK will automatically read it from the GOOGLE_API_KEY environment variable.
    # In Colab get the key from Colab-secrets ("🔑" in the left panel).
    import os
    from google.colab import userdata

    os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")
except ImportError:
    pass

import google.generativeai as genai

# Parse the arguments

model = 'gemini-1.5-flash' # @param {isTemplate: true}
contents_b64 = 'W3sicm9sZSI6InVzZXIiLCJwYXJ0cyI6W3sidGV4dCI6Im9ubGluZSBidXNpbmVzcyBpZGVhcyAyMDI0In1dfSx7InJvbGUiOiJtb2RlbCIsInBhcnRzIjpbeyJ0ZXh0IjoiIyMgT25saW5lIEJ1c2luZXNzIElkZWFzIGZvciAyMDI0OlxcblxcbkhlcmUgYXJlIHNvbWUgb25saW5lIGJ1c2luZXNzIGlkZWFzIGZvciAyMDI0LCBjYXRlZ29yaXplZCBieSB0aGVpciBmb2N1czpcXG5cXG4qKkUtY29tbWVyY2UgJiBSZXRhaWw6KipcXG5cXG4qICoqU3VzdGFpbmFibGUgJiBFdGhpY2FsIFByb2R1Y3RzOioqIE9mZmVyIGVjby1mcmllbmRseSwgZXRoaWNhbGx5IHNvdXJjZWQgY2xvdGhpbmcsIGhvbWUgZ29vZHMsIG9yIGJlYXV0eSBwcm9kdWN0cy5cXG4qICoqUGVyc29uYWxpemVkIFByb2R1Y3RzOioqIE9mZmVyIGN1c3RvbWl6ZWQgZ2lmdHMsIGNsb3RoaW5nLCBvciBhY2Nlc3Nvcmllcy5cXG4qICoqU3Vic2NyaXB0aW9uIEJveGVzOioqIEZvY3VzIG9uIG5pY2hlIGludGVyZXN0cyBsaWtlIHBldCBzdXBwbGllcywgYmVhdXR5IHByb2R1Y3RzLCBvciBoZWFsdGh5IHNuYWNrcy5cXG4qICoqRHJvcHNoaXBwaW5nOioqIFNlbGwgcHJvZHVjdHMgb25saW5lIHdpdGhvdXQgaG9sZGluZyBpbnZlbnRvcnkuXFxuKiAqKlJlc2VsbGluZzoqKiBTb3VyY2UgdW5pcXVlIGl0ZW1zIGZyb20gdGhyaWZ0IHN0b3JlcyBvciBvbmxpbmUgbWFya2V0cGxhY2VzIGFuZCByZXNlbGwgdGhlbSBmb3IgcHJvZml0LlxcbiogKipVcGN5Y2xlZCAmIFZpbnRhZ2U6KiogU2VsbCB2aW50YWdlIGNsb3RoaW5nLCBmdXJuaXR1cmUsIG9yIGhvbWUgZGVjb3Igd2l0aCBhIG1vZGVybiB0d2lzdC5cXG5cXG4qKkRpZ2l0YWwgU2VydmljZXM6KipcXG5cXG4qICoqVmlydHVhbCBBc3Npc3RhbnQ6KiogT2ZmZXIgYWRtaW5pc3RyYXRpdmUsIHRlY2huaWNhbCwgb3IgY3JlYXRpdmUgc3VwcG9ydCB0byBidXNpbmVzc2VzIGFuZCBpbmRpdmlkdWFscy5cXG4qICoqU29jaWFsIE1lZGlhIE1hbmFnZW1lbnQ6KiogTWFuYWdlIHNvY2lhbCBtZWRpYSBhY2NvdW50cyBmb3IgYnVzaW5lc3NlcyBhbmQgaW5kaXZpZHVhbHMuXFxuKiAqKldlYiBEZXNpZ24gJiBEZXZlbG9wbWVudDoqKiBDcmVhdGUgd2Vic2l0ZXMgYW5kIG9ubGluZSBhcHBsaWNhdGlvbnMuXFxuKiAqKkNvbnRlbnQgQ3JlYXRpb246KiogV3JpdGUgYXJ0aWNsZXMsIGJsb2cgcG9zdHMsIHdlYnNpdGUgY29weSwgc29jaWFsIG1lZGlhIHBvc3RzLCBvciB2aWRlb3MuXFxuKiAqKk9ubGluZSBUdXRvcmluZzoqKiBQcm92aWRlIHR1dG9yaW5nIHNlcnZpY2VzIGluIHZhcmlvdXMgc3ViamVjdHMuXFxuKiAqKkdyYXBoaWMgRGVzaWduOioqIERlc2lnbiBsb2dvcywgYnJhbmRpbmcgbWF0ZXJpYWxzLCBhbmQgb3RoZXIgdmlzdWFsIGFzc2V0cy5cXG4qICoqRnJlZWxhbmNlIFdyaXRpbmc6KiogT2ZmZXIgd3JpdGluZyBzZXJ2aWNlcyBmb3Igd2Vic2l0ZXMsIGJsb2dzLCBhbmQgb3RoZXIgcGxhdGZvcm1zLlxcblxcbioqT3RoZXI6KipcXG5cXG4qICoqT25saW5lIENvdXJzZSBDcmVhdGlvbjoqKiBUZWFjaCBhIHNraWxsIG9yIHNoYXJlIHlvdXIga25vd2xlZGdlIHRocm91Z2ggYW4gb25saW5lIGNvdXJzZS5cXG4qICoqQWZmaWxpYXRlIE1hcmtldGluZzoqKiBQcm9tb3RlIHByb2R1Y3RzIG9yIHNlcnZpY2VzIGFuZCBlYXJuIGEgY29tbWlzc2lvbiBvbiBzYWxlcy5cXG4qICoqT25saW5lIENvYWNoaW5nOioqIFByb3ZpZGUgY29hY2hpbmcgc2VydmljZXMgaW4gYXJlYXMgbGlrZSBmaXRuZXNzLCBudXRyaXRpb24sIG9yIHBlcnNvbmFsIGRldmVsb3BtZW50LlxcbiogKipEaWdpdGFsIE1hcmtldGluZyBBZ2VuY3k6KiogT2ZmZXIgc2VydmljZXMgbGlrZSBTRU8sIHBhaWQgYWR2ZXJ0aXNpbmcsIGFuZCBzb2NpYWwgbWVkaWEgbWFya2V0aW5nLlxcbiogKipPbmxpbmUgQ29tbXVuaXR5IEJ1aWxkaW5nOioqIENyZWF0ZSBhIHBsYXRmb3JtIGZvciBwZW9wbGUgd2l0aCBzaGFyZWQgaW50ZXJlc3RzLlxcbiogKipFLWJvb2tzICYgRGlnaXRhbCBQcm9kdWN0czoqKiBDcmVhdGUgYW5kIHNlbGwgZS1ib29rcywgdGVtcGxhdGVzLCBvciBvdGhlciBkaWdpdGFsIHByb2R1Y3RzLlxcbiogKipORlQgQ3JlYXRpb24gJiBTZWxsaW5nOioqIENyZWF0ZSBhbmQgc2VsbCBub24tZnVuZ2libGUgdG9rZW5zIChORlRzKSBvbiB0aGUgYmxvY2tjaGFpbi5cXG5cXG4qKkVtZXJnaW5nIFRyZW5kczoqKlxcblxcbiogKipBSSAmIE1hY2hpbmUgTGVhcm5pbmc6KiogRGV2ZWxvcCBBSS1wb3dlcmVkIHNvbHV0aW9ucyBmb3IgYnVzaW5lc3NlcyBvciBjb25zdW1lcnMuXFxuKiAqKldlYjMgJiBCbG9ja2NoYWluOioqIEJ1aWxkIGRlY2VudHJhbGl6ZWQgYXBwbGljYXRpb25zIChEQXBwcykgb3Igb2ZmZXIgc2VydmljZXMgcmVsYXRlZCB0byB0aGUgYmxvY2tjaGFpbi5cXG4qICoqTWV0YXZlcnNlOioqIERldmVsb3Agb3Igc2VsbCB2aXJ0dWFsIGdvb2RzIGFuZCBzZXJ2aWNlcyBmb3IgbWV0YXZlcnNlIHBsYXRmb3Jtcy5cXG4qICoqU3VzdGFpbmFibGUgJiBFY28tRnJpZW5kbHkgU29sdXRpb25zOioqIE9mZmVyIGVudmlyb25tZW50YWxseSBjb25zY2lvdXMgcHJvZHVjdHMgb3Igc2VydmljZXMuXFxuKiAqKkhlYWx0aCAmIFdlbGxuZXNzOioqIEZvY3VzIG9uIG9ubGluZSBzZXJ2aWNlcyByZWxhdGVkIHRvIG1lbnRhbCBoZWFsdGgsIGZpdG5lc3MsIG9yIG51dHJpdGlvbi5cXG5cXG4qKlRpcHMgZm9yIFN1Y2Nlc3M6KipcXG5cXG4qICoqRm9jdXMgb24gYSBuaWNoZToqKiBJZGVudGlmeSBhIHNwZWNpZmljIG1hcmtldCB3aXRoIHVubWV0IG5lZWRzLlxcbiogKipCdWlsZCBhIHN0cm9uZyBvbmxpbmUgcHJlc2VuY2U6KiogQ3JlYXRlIGEgcHJvZmVzc2lvbmFsIHdlYnNpdGUgYW5kIGVuZ2FnZSBvbiBzb2NpYWwgbWVkaWEuXFxuKiAqKlByb3ZpZGUgZXhjZWxsZW50IGN1c3RvbWVyIHNlcnZpY2U6KiogQnVpbGQgcmVsYXRpb25zaGlwcyB3aXRoIGN1c3RvbWVycyBhbmQgZ28gdGhlIGV4dHJhIG1pbGUuXFxuKiAqKlN0YXkgdXAtdG8tZGF0ZSB3aXRoIHRyZW5kczoqKiBDb250aW51b3VzbHkgbGVhcm4gYW5kIGFkYXB0IHRvIHRoZSBldmVyLWNoYW5naW5nIG9ubGluZSBsYW5kc2NhcGUuXFxuKiAqKk1hcmtldCB5b3VyIGJ1c2luZXNzIGVmZmVjdGl2ZWx5OioqIFVzZSBkaWdpdGFsIG1hcmtldGluZyBzdHJhdGVnaWVzIHRvIHJlYWNoIHlvdXIgdGFyZ2V0IGF1ZGllbmNlLlxcbiogKipEb24ndCBiZSBhZnJhaWQgdG8gZXhwZXJpbWVudDoqKiBUcnkgbmV3IHRoaW5ncyBhbmQgbGVhcm4gZnJvbSB5b3VyIG1pc3Rha2VzLlxcblxcblJlbWVtYmVyLCB0aGUgYmVzdCBvbmxpbmUgYnVzaW5lc3MgaWRlYSBmb3IgeW91IHdpbGwgZGVwZW5kIG9uIHlvdXIgc2tpbGxzLCBpbnRlcmVzdHMsIGFuZCByZXNvdXJjZXMuIERvIHlvdXIgcmVzZWFyY2gsIGlkZW50aWZ5IHlvdXIgdGFyZ2V0IG1hcmtldCwgYW5kIGNyZWF0ZSBhIHVuaXF1ZSB2YWx1ZSBwcm9wb3NpdGlvbi4gV2l0aCBoYXJkIHdvcmsgYW5kIGRlZGljYXRpb24sIHlvdSBjYW4gYnVpbGQgYSBzdWNjZXNzZnVsIG9ubGluZSBidXNpbmVzcyBpbiAyMDI0LiBcXG4ifV19XQ==' # @param {isTemplate: true}
generation_config_b64 = 'eyJ0ZW1wZXJhdHVyZSI6MSwidG9wX3AiOjAuOTUsInRvcF9rIjo0MCwibWF4X291dHB1dF90b2tlbnMiOjgxOTJ9' # @param {isTemplate: true}
safety_settings_b64 = "e30="  # @param {isTemplate: true}

gais_contents = json.loads(base64.b64decode(contents_b64))

generation_config = json.loads(base64.b64decode(generation_config_b64))
safety_settings = json.loads(base64.b64decode(safety_settings_b64))

stream = False

# Convert and upload the files

tempfiles = pathlib.Path(f"tempfiles")
tempfiles.mkdir(parents=True, exist_ok=True)


drive = None
def upload_file_data(file_data, index):
    """Upload files to the Files API.

    For each file, Google AI Studio either sent:
    - a Google Drive ID,
    - a URL,
    - a file path, or
    - The raw bytes (`inline_data`).

    The API only understands `inline_data` or it's Files API.
    This code, uploads files to the files API where the API can access them.
    """

    mime_type = file_data["mime_type"]
    if drive_id := file_data.pop("drive_id", None):
        if drive is None:
          from google.colab import drive
          drive.mount("/gdrive")

        path = next(
            pathlib.Path(f"/gdrive/.shortcut-targets-by-id/{drive_id}").glob("*")
        )
        print("Uploading:", str(path))
        file_info = genai.upload_file(path=path, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if url := file_data.pop("url", None):
        response = requests.get(url)
        data = response.content
        name = url.split("/")[-1]
        path = tempfiles / str(index)
        path.write_bytes(data)
        print("Uploading:", url)
        file_info = genai.upload_file(path, display_name=name, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if name := file_data.get("filename", None):
        if not pathlib.Path(name).exists():
            raise IOError(
                f"local file: `{name}` does not exist. You can upload files "
                'to Colab using the file manager ("📁 Files" in the left '
                "toolbar)"
            )
        file_info = genai.upload_file(path, display_name=name, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if "inline_data" in file_data:
        return

    raise ValueError("Either `drive_id`, `url` or `inline_data` must be provided.")


contents = copy.deepcopy(gais_contents)

index = 0
for content in contents:
    for n, part in enumerate(content["parts"]):
        if file_data := part.get("file_data", None):
            upload_file_data(file_data, index)
            index += 1

import json
print(json.dumps(contents, indent=4))

[
    {
        "role": "user",
        "parts": [
            {
                "text": "online business ideas 2024"
            }
        ]
    },
    {
        "role": "model",
        "parts": [
            {
                "text": "## Online Business Ideas for 2024:\\n\\nHere are some online business ideas for 2024, categorized by their focus:\\n\\n**E-commerce & Retail:**\\n\\n* **Sustainable & Ethical Products:** Offer eco-friendly, ethically sourced clothing, home goods, or beauty products.\\n* **Personalized Products:** Offer customized gifts, clothing, or accessories.\\n* **Subscription Boxes:** Focus on niche interests like pet supplies, beauty products, or healthy snacks.\\n* **Dropshipping:** Sell products online without holding inventory.\\n* **Reselling:** Source unique items from thrift stores or online marketplaces and resell them for profit.\\n* **Upcycled & Vintage:** Sell vintage clothing, furniture, or home decor with a modern twist.\\n\\n**Digital Services:**\\n\

## Call `generate_content`

In [3]:
from IPython.display import display
from IPython.display import Markdown

# Call the model and print the response.
gemini = genai.GenerativeModel(model_name=model)

response = gemini.generate_content(
    contents,
    generation_config=generation_config,
    safety_settings=safety_settings,
    stream=stream,
)

display(Markdown(response.text))

**Bonus:**

* **Consider using tools:** There are many tools available to help you with your online business, including website builders, social media management platforms, email marketing tools, and more.
* **Network with other entrepreneurs:** Connect with other online business owners to share tips, resources, and support.\n
* **Don't give up!** Starting and growing an online business takes time and effort. Stay persistent and you will eventually achieve your goals.

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://ai.google.dev/gemini-api/docs"><img src="https://ai.google.dev/static/site-assets/images/docs/notebook-site-button.png" height="32" width="32" />Docs on ai.google.dev</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/google-gemini/cookbook/blob/main/quickstarts"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />More notebooks in the Cookbook</a>
  </td>
</table>

## [optional] Show the conversation

This section displays the conversation received from Google AI Studio.

In [ ]:
# @title Show the conversation, in colab.
import mimetypes

def show_file(file_data):
    mime_type = file_data["mime_type"]

    if drive_id := file_data.get("drive_id", None):
        path = next(
            pathlib.Path(f"/gdrive/.shortcut-targets-by-id/{drive_id}").glob("*")
        )
        name = path
        # data = path.read_bytes()
        kwargs = {"filename": path}
    elif url := file_data.get("url", None):
        name = url
        kwargs = {"url": url}
        # response = requests.get(url)
        # data = response.content
    elif data := file_data.get("inline_data", None):
        name = None
        kwargs = {"data": data}
    elif name := file_data.get("filename", None):
        if not pathlib.Path(name).exists():
            raise IOError(
                f"local file: `{name}` does not exist. You can upload files to "
                'Colab using the file manager ("📁 Files"in the left toolbar)'
            )
    else:
        raise ValueError("Either `drive_id`, `url` or `inline_data` must be provided.")

        print(f"File:\n    name: {name}\n    mime_type: {mime_type}\n")
        return

    format = mimetypes.guess_extension(mime_type).strip(".")
    if mime_type.startswith("image/"):
        image = IPython.display.Image(**kwargs, width=256)
        IPython.display.display(image)
        print()
        return

    if mime_type.startswith("audio/"):
        if len(data) < 2**12:
            audio = IPython.display.Audio(**kwargs)
            IPython.display.display(audio)
            print()
            return

    if mime_type.startswith("video/"):
        if len(data) < 2**12:
            audio = IPython.display.Video(**kwargs, mimetype=mime_type)
            IPython.display.display(audio)
            print()
            return

    print(f"File:\n    name: {name}\n    mime_type: {mime_type}\n")


for content in gais_contents:
    if role := content.get("role", None):
        print("Role:", role, "\n")

    for n, part in enumerate(content["parts"]):
        if text := part.get("text", None):
            print(text, "\n")

        elif file_data := part.get("file_data", None):
            show_file(file_data)

    print("-" * 80, "\n")